In [141]:
import brightway2 as bw
import pandas as pd
import numpy as np

In [2]:
bw.projects.set_current('ecoinvent3.8')

Ecoinvent contains grid mixes for the U.S. electric regions listed in the cell below.

Source: Ecoinvent electricity writeup at
https://proyectaryproducir.com.ar/public_html/Seminarios_Posgrado/Material_de_referencia/ecoinvent-electricity%20datasets%20201305%20final.pdf

In [8]:
elec_locs = ['FRCC', 'ASCC', 'HICC', 
             'MRO', 'NPCC', 'RFC', 
             'SERC', 'SPP', 'TRE',
             'WECC']

In [78]:
elec = [i for i in bw.Database('ecoinvent3.7.1 cut-off') 
             for loc in elec_locs 
             if (loc in i.as_dict()['location'])
        and ('electricity' in i.as_dict()['name'])]

In [79]:
elec

['electricity production, lignite' (kilowatt hour, US-RFC, None),
 'electricity production, lignite' (kilowatt hour, US-SERC, None),
 'market for electricity, low voltage' (kilowatt hour, US-FRCC, None),
 'electricity, high voltage, production mix' (kilowatt hour, US-WECC, None),
 'electricity production, wind, <1MW turbine, onshore' (kilowatt hour, US-NPCC, None),
 'electricity production, nuclear, boiling water reactor' (kilowatt hour, US-SERC, None),
 'electricity production, natural gas, conventional power plant' (kilowatt hour, US-WECC, None),
 'electricity production, nuclear, boiling water reactor' (kilowatt hour, US-WECC, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, US-MRO, None),
 'electricity production, lignite' (kilowatt hour, US-TRE, None),
 'electricity, high voltage, production mix' (kilowatt hour, US-RFC, None),
 'electricity production, photovoltaic, 570kWp open ground installation, multi-Si' (kilowatt hour, US-HICC, None),
 'electrici

We ignore "production mix" processes because they "are only of informative
character and it is NOT recommended to use the production mixes in your life
cycle analysis, because the electricity markets represent electricity supply" (per source above).

There are "market for electricity" and "market group" processes. The "market groups" combine grid mixes from the various U.S. regions into a national grid mix for use as an input to U.S. activities.

In [82]:
elecmarketgroup = [i for i in bw.Database('ecoinvent3.7.1 cut-off') 
             if ('US' in i.as_dict()['location'])
             and ('market group for electricity' in i.as_dict()['name'])]

In [83]:
elecmarketgroup

['market group for electricity, low voltage' (kilowatt hour, US, None),
 'market group for electricity, high voltage' (kilowatt hour, US, None),
 'market group for electricity, medium voltage' (kilowatt hour, US, None)]

The "market for electricity" processes will tell us the grid mix.

In [80]:
elecmixes = [i for i in bw.Database('ecoinvent3.7.1 cut-off') 
             for loc in elec_locs 
             if (loc in i.as_dict()['location'])
             and ('market for electricity' in i.as_dict()['name'])]

In [81]:
elecmixes

['market for electricity, low voltage' (kilowatt hour, US-MRO, None),
 'market for electricity, low voltage' (kilowatt hour, US-WECC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-SERC, None),
 'market for electricity, high voltage' (kilowatt hour, US-HICC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-MRO, None),
 'market for electricity, high voltage' (kilowatt hour, US-RFC, None),
 'market for electricity, low voltage' (kilowatt hour, US-NPCC, None),
 'market for electricity, high voltage' (kilowatt hour, US-MRO, None),
 'market for electricity, high voltage' (kilowatt hour, US-SERC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-NPCC, None),
 'market for electricity, low voltage' (kilowatt hour, US-ASCC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-TRE, None),
 'market for electricity, low voltage' (kilowatt hour, US-RFC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-ASCC, 

What do the exchanges look like for these "market for electricity" processes? 

Low voltage process.

In [84]:
mix1 = elecmixes[0]
mix1

'market for electricity, low voltage' (kilowatt hour, US-MRO, None)

In [85]:
[i for i in bw.Database('ecoinvent3.7.1 cut-off').get(mix1.key[1]).exchanges()]

[Exchange: 7.79e-09 kilogram 'market for sulfur hexafluoride, liquid' (kilogram, RoW, None) to 'market for electricity, low voltage' (kilowatt hour, US-MRO, None)>,
 Exchange: 8.74048809653223e-08 kilometer 'market for distribution network, electricity, low voltage' (kilometer, GLO, None) to 'market for electricity, low voltage' (kilowatt hour, US-MRO, None)>,
 Exchange: 1.0 kilowatt hour 'market for electricity, low voltage' (kilowatt hour, US-MRO, None) to 'market for electricity, low voltage' (kilowatt hour, US-MRO, None)>,
 Exchange: 0.00166388556322942 kilowatt hour 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, RoW, None) to 'market for electricity, low voltage' (kilowatt hour, US-MRO, None)>,
 Exchange: 0.00133598877177478 kilowatt hour 'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted' (kilowatt hour, RoW, None) to 'market for electricity, low voltage' (kilowatt hour

In this low-voltage process, there's a tiny fraction of solar electricity being added to electricity that's converted from medium voltage. The solar electricity is coming from two types of roof installations and a multi-SI ground installation. Total electricity from these additional generating processes is less than 0.006 kWh per kWh of low voltage electricity.

Medium voltage process.

In [86]:
mix2 = elecmixes[2]
mix2

'market for electricity, medium voltage' (kilowatt hour, US-SERC, None)

In [90]:
[i for i in bw.Database('ecoinvent3.7.1 cut-off').get(mix2.key[1]).exchanges()]

[Exchange: 1.86277676887616e-08 kilometer 'market for transmission network, electricity, medium voltage' (kilometer, GLO, None) to 'market for electricity, medium voltage' (kilowatt hour, US-SERC, None)>,
 Exchange: 1.29e-07 kilogram 'market for sulfur hexafluoride, liquid' (kilogram, RoW, None) to 'market for electricity, medium voltage' (kilowatt hour, US-SERC, None)>,
 Exchange: 1.0 kilowatt hour 'market for electricity, medium voltage' (kilowatt hour, US-SERC, None) to 'market for electricity, medium voltage' (kilowatt hour, US-SERC, None)>,
 Exchange: 0.998990651687093 kilowatt hour 'electricity voltage transformation from high to medium voltage' (kilowatt hour, US-SERC, None) to 'market for electricity, medium voltage' (kilowatt hour, US-SERC, None)>,
 Exchange: 0.0010093483129066 kilowatt hour 'electricity, from municipal waste incineration to generic market for electricity, medium voltage' (kilowatt hour, RoW, None) to 'market for electricity, medium voltage' (kilowatt hour, US

At medium voltage, there's again some electricity generation being added to electricity converted from high voltage. The new generation is only from municipal waste incineration, 0.001 kWh per kWh medium voltage electricity. There is also some parasitic consumption, 0.003 kWh per kWh.

High voltage process.

In [91]:
mix3 = elecmixes[5]
mix3

'market for electricity, high voltage' (kilowatt hour, US-RFC, None)

In [163]:
ex = [i for i in bw.Database('ecoinvent3.7.1 cut-off').get(mix3.key[1]).exchanges()]

In [168]:
ex

[Exchange: 1.0 kilowatt hour 'market for electricity, high voltage' (kilowatt hour, US-RFC, None) to 'market for electricity, high voltage' (kilowatt hour, US-RFC, None)>,
 Exchange: 6.58209848825026e-09 kilometer 'market for transmission network, electricity, high voltage' (kilometer, GLO, None) to 'market for electricity, high voltage' (kilowatt hour, US-RFC, None)>,
 Exchange: 3.17e-10 kilometer 'market for transmission network, long-distance' (kilometer, GLO, None) to 'market for electricity, high voltage' (kilowatt hour, US-RFC, None)>,
 Exchange: 0.288515691683293 kilowatt hour 'electricity production, hard coal' (kilowatt hour, US-RFC, None) to 'market for electricity, high voltage' (kilowatt hour, US-RFC, None)>,
 Exchange: 0.00255768100056629 kilowatt hour 'electricity production, hydro, reservoir, alpine region' (kilowatt hour, US-RFC, None) to 'market for electricity, high voltage' (kilowatt hour, US-RFC, None)>,
 Exchange: 0.0102307731738715 kilowatt hour 'electricity produ

The high voltage process is where the vast majority of the grid mix definition takes place. In this process, the following power generator types are used (fuels or renewables):

In [175]:
exkey = [i.as_dict()['input'][1] for i in bw.Database('ecoinvent3.7.1 cut-off').get(mix3.key[1]).exchanges()]
for i in list(np.arange(len(ex))):
    print(i, bw.Database('ecoinvent3.7.1 cut-off').get(exkey[i]))

0 'market for electricity, high voltage' (kilowatt hour, US-RFC, None)
1 'market for transmission network, electricity, high voltage' (kilometer, GLO, None)
2 'market for transmission network, long-distance' (kilometer, GLO, None)
3 'electricity production, hard coal' (kilowatt hour, US-RFC, None)
4 'electricity production, hydro, reservoir, alpine region' (kilowatt hour, US-RFC, None)
5 'electricity production, hydro, run-of-river' (kilowatt hour, US-RFC, None)
6 'electricity production, lignite' (kilowatt hour, US-RFC, None)
7 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, US-RFC, None)
8 'electricity production, natural gas, conventional power plant' (kilowatt hour, US-RFC, None)
9 'electricity production, nuclear, boiling water reactor' (kilowatt hour, US-RFC, None)
10 'electricity production, nuclear, pressure water reactor' (kilowatt hour, US-RFC, None)
11 'electricity production, oil' (kilowatt hour, US-RFC, None)
12 'electricity production, wi

ActivityDatasetDoesNotExist: <Model: ActivityDataset> instance matching query does not exist:
SQL: SELECT "t1"."id", "t1"."data", "t1"."code", "t1"."database", "t1"."location", "t1"."name", "t1"."product", "t1"."type" FROM "activitydataset" AS "t1" WHERE (("t1"."database" = ?) AND ("t1"."code" = ?)) ORDER BY Random() LIMIT ? OFFSET ?
Params: ['ecoinvent3.7.1 cut-off', '20185046-64bb-4c09-a8e7-e8a9e144ca98', 1, 0]

In [72]:
market1=elecmarketgroup[0]
[i for i in bw.Database('ecoinvent3.7.1 cut-off').get(market1.key[1]).exchanges()]

[Exchange: 1.0 kilowatt hour 'market group for electricity, high voltage' (kilowatt hour, US, None) to 'market group for electricity, high voltage' (kilowatt hour, US, None)>,
 Exchange: 0.00128604683938465 kilowatt hour 'market for electricity, high voltage' (kilowatt hour, US-ASCC, None) to 'market group for electricity, high voltage' (kilowatt hour, US, None)>,
 Exchange: 0.0546860566892221 kilowatt hour 'market for electricity, high voltage' (kilowatt hour, US-FRCC, None) to 'market group for electricity, high voltage' (kilowatt hour, US, None)>,
 Exchange: 0.00215847303397079 kilowatt hour 'market for electricity, high voltage' (kilowatt hour, US-HICC, None) to 'market group for electricity, high voltage' (kilowatt hour, US, None)>,
 Exchange: 0.112963252700324 kilowatt hour 'market for electricity, high voltage' (kilowatt hour, US-MRO, None) to 'market group for electricity, high voltage' (kilowatt hour, US, None)>,
 Exchange: 0.0632203796486565 kilowatt hour 'market for electric

In [73]:
elecfuel = [i for i in bw.Database('ecoinvent3.7.1 cut-off') 
             for loc in elec_locs 
             if (loc in i.as_dict()['location'])
             and ('electricity production' in i.as_dict()['name'])]
elecfuel

['electricity production, nuclear, pressure water reactor' (kilowatt hour, US-RFC, None),
 'electricity production, nuclear, boiling water reactor' (kilowatt hour, US-SERC, None),
 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, US-ASCC, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, US-NPCC, None),
 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, US-SERC, None),
 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, US-NPCC, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, US-TRE, None),
 'electricity production, lignite' (kilowatt hour, US-NPCC, None),
 'electricity production, deep geothermal' (kilowatt hour, US-HICC, None),
 'electricity production, lignite' (kilowatt hour, US-HICC, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, US-AS

In [60]:
elecconsumer = usdb[1]
elecconsumer

'beryllium hydroxide production' (kilogram, US, None)

In [61]:
[i for i in bw.Database('ecoinvent3.7.1 cut-off').get(elecconsumer.key[1]).exchanges()]

[Exchange: 15.93291405 megajoule 'market for heat, district or industrial, natural gas' (megajoule, RoW, None) to 'beryllium hydroxide production' (kilogram, US, None)>,
 Exchange: -0.733199502 cubic meter 'market for wastewater from pig iron production' (cubic meter, RoW, None) to 'beryllium hydroxide production' (kilogram, US, None)>,
 Exchange: 1.0 kilogram 'beryllium hydroxide production' (kilogram, US, None) to 'beryllium hydroxide production' (kilogram, US, None)>,
 Exchange: 6.9392e-06 meter 'market for conveyor belt' (meter, GLO, None) to 'beryllium hydroxide production' (kilogram, US, None)>,
 Exchange: 1.551362683 kilogram 'market for sulfuric acid' (kilogram, RoW, None) to 'beryllium hydroxide production' (kilogram, US, None)>,
 Exchange: 4.57023e-09 unit 'market for mine infrastructure, open cast, non-ferrous metal' (unit, GLO, None) to 'beryllium hydroxide production' (kilogram, US, None)>,
 Exchange: 15.93291405 kilowatt hour 'market group for electricity, medium voltage'

For reference: A list of all processes in Ecoinvent with a US location. These processes will use a "market group for electricity" input that combines "market for electricity" processes and ultimately uses U.S. grid mixes.

In [ ]:
usdb = [i for i in bw.Database('ecoinvent3.7.1 cut-off') if (
    i.as_dict()['location'] == 'US'
)]